# HW 2
## Titanic Survival Predictions
In this dataset, you will train a logistic regression model that can predict the survivors based on a set of features about the passenger. Start first by taking some time to understand your data. You can find more info about the data and its breakdown in this link https://www.kaggle.com/c/titanic/data

### Start with Data Exploring
Your data is split into two files. One for training and one for testing. Load both and find out the number of samples and features in each file

In [1]:
import pandas as pd
import numpy as np
import pylab as plt

df = pd.read_csv('train.csv')
df.head()

FileNotFoundError: ignored

In [ ]:

df1 = pd.read_csv('test.csv')
df1.head()

In [ ]:
df.isnull().sum()
print("Train Shape:",df.shape)
df1.isnull().sum()
print("Test Shape:",df1.shape)

How many survivors in the training set?

Can you find Jack and Rose in the data? Use some code. You probably won't find them as they were fictional characters. See if you can find Beatrice Irene Sandstorm? Read here about her https://www.encyclopedia-titanica.org/titanic-survivor/beatrice-irene-sandstrom.html

In [ ]:
boolean_findings = df['Name'].str.contains('Beatrice Irene Sandstorm')
boolean_findings

total_occurence = boolean_findings.sum()

if(total_occurence > 0):
    print("yes the string is present in the column")
else:
    print("The Name is not in the data")

### Does features make a difference?
In the training dataset (file)
- Pclass - Usually, higher class (class 1) meant a more expensive fare and the other classes paid lesser to be on the ship. What was the survival rate per class?
- Sex — What was the survival rate of women?
- SibSp — What was the survival rate based on number of siblings the passenger had on the ship?
- Parch — What was the survival rate based on number of parents/children the passenger had on the ship?

In [ ]:
plt.rc('figure', figsize=(10, 5))

# Size of matplotlib figures that contain subplots
fizsize_with_subplots = (10, 10)

# Size of matplotlib histogram bins
bin_size = 10

In [ ]:
def bar_chart(feature):
    survived = df[df['Survived']==1][feature].value_counts()
    dead = df[df['Survived']==0][feature].value_counts()
    data = pd.DataFrame([survived,dead])
    data.index = ['Survived','Dead']
    data.plot(kind='bar',stacked=True, figsize=(10,5))

## Pclass

In [ ]:
bar_chart('Pclass')
print("Survived :\n",df[df['Survived']==1]['Pclass'].value_counts())
print("Dead:\n",df[df['Survived']==0]['Pclass'].value_counts())

# sex

In [ ]:
bar_chart('Sex')
print("Survived :\n",df[df['Survived']==1]['Sex'].value_counts())
print("Dead:\n",df[df['Survived']==0]['Sex'].value_counts())

# sibSp

In [ ]:
bar_chart('SibSp')
print("Survived :\n",df[df['Survived']==1]['SibSp'].value_counts())
print("Dead:\n",df[df['Survived']==0]['SibSp'].value_counts())

The Chart confirms a person aboarded with more than 2 siblings or spouse more likely survived.
The Chart confirms a person aboarded without siblings or spouse more likely dead

In [ ]:
bar_chart('Parch')
print("Survived :\n",df[df['Survived']==1]['Parch'].value_counts())
print("Dead:\n",df[df['Survived']==0]['Parch'].value_counts())

The Chart confirms a person aboarded with more than 2 parents or children more likely survived.
The Chart confirms a person aboarded alone more likely dead

Plot two histogram based on the age. One for the survivors and other for non survivors. According to the histogram was Jack more likely to survive or not survive and why?

In [ ]:
df['AgeFill'] = df['Age']


In [ ]:
df['AgeFill'] = df['AgeFill'] \
                        .groupby([df['Sex'], df['Pclass']]) \
                        .apply(lambda x: x.fillna(x.median()))

In [ ]:
len(df[df['AgeFill'].isnull()])

In [ ]:
max_age = max(df['AgeFill'])
max_age

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=fizsize_with_subplots)
n_equal_bins = 1
# Histogram of AgeFill segmented by Survived
data1 = df[df['Survived'] == 0]['Age']
data2 = df[df['Survived'] == 1]['Age']
max_age = max(df['AgeFill'])
axes[0].hist([data1, data2],
             bins= 8,  #bin = max_age / bin_size
             range=(1, max_age),
             stacked=True)
axes[0].legend(('Died', 'Survived'), loc='best')
axes[0].set_title('Survivors by Age Groups Histogram')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Count')

# Scatter plot Survived and AgeFill
axes[1].scatter(df['Survived'], df['AgeFill'])
axes[1].set_title('Survivors by Age Plot')
axes[1].set_xlabel('Survived')
axes[1].set_ylabel('Age')

Unfortunately, the graphs above do not seem to clearly show any insights

Divide the age histograms into 3 X 2 subplots. Left column is for non survival data and the right one is for survivors. Each row is for one of the three PClasses

In [ ]:
df.describe()

NameError: ignored

In [ ]:
# Set up a grid of plots
fig = plt.figure(figsize=fizsize_with_subplots)
fig_dims = (3, 2)

# Plot death and survival counts
plt.subplot2grid(fig_dims, (0, 0))
df['Survived'].value_counts().plot(kind='bar',
                                         title='Death and Survival Counts')

# Plot Pclass counts
plt.subplot2grid(fig_dims, (0, 1))
df['Pclass'].value_counts().plot(kind='bar',
                                       title='Passenger Class Counts')

# Plot Sex counts
plt.subplot2grid(fig_dims, (1, 0))
df['Sex'].value_counts().plot(kind='bar',
                                    title='Gender Counts')
plt.xticks(rotation=0)

# Plot Embarked counts
plt.subplot2grid(fig_dims, (1, 1))
df['Embarked'].value_counts().plot(kind='bar',
                                         title='embark Counts')

# Plot the Age histogram
plt.subplot2grid(fig_dims, (2, 0))
df['Age'].hist()
plt.title('Age Histogram')

## Fix missing Data in the Training Dataset
Which features in the training contains missing values? and how many missing values are there?

In [ ]:
df.info()

In [ ]:
df[df['Embarked'].isnull()]

In [ ]:
df[df['Age'].isnull()]

In [ ]:
df[df['Cabin'].isnull()]

In [ ]:
 print(" \nCount total NaN in a DataFrame : \n\n",
       df.isnull().sum().sum())

Find out the median age grouped by PClass and gender. Clean up the data by substituting the nans by the median from the corresponding PClass and gender

In [ ]:
df.groupby(["Sex",'Pclass']).median()

The median age grouped by PClass and gender is 31.0 

In [ ]:
df[['Sex','Pclass']]=df[['Sex','Pclass']].fillna(df.median().iloc[0])
df[['Sex','Pclass']]

For the Embarked column, substitute the missing value with the highest occurence value among the entire dataset

In [ ]:
df['Embarked']=df['Embarked'].fillna(df.mode().iloc[0])
df['Embarked']

## Let's build our Model
Cabin, Name, ticket and Passenger Id are not important to our ML model. Let's get rid of them

In [ ]:
df.info()

In [ ]:
df= df.drop(labels=['Name', 'PassengerId', 'Ticket', 'Cabin'], axis= 1)
df

Make sure now that your data does not have any nulls

In [ ]:
df.dtypes[df.dtypes.map(lambda x: x == 'object')]

In [ ]:
df.dropna()

In [ ]:
print(" \nCount total NaN in a DataFrame : \n\n",
      df.isnull().sum().sum())

We need to convert some of our features to categories and numbers instead of letters as logistic regression model won't be able to decode them. Map the embarked variable into 0,1, and 2 and the sex variable into 0 and 1

% hint: `df_train['Embarked'].astype('category').cat.codes`

In [ ]:
df['Embarked'].astype('category').cat.codes

In [ ]:
df['Sex'].astype('category').cat.codes

You are now ready to build a logistic regression model and fit it. You do not to split your data since I have already splitted for you in 2 different files so use all the training

Show the training score. 

What are the coefficients. Make a plot that shows the importance of each feature in your model

Use the logistic regression model on the test data to predict who will survive. You may need to do same cleaning you have done in the training data to the test data

How many will survive from the test dataset?

# Chapter 3 exercises

**A.** Try to build a classifier for the MNIST dataset that achieves over 97% accuracy on the test set. Hint: the `KNeighborsClassifier` works quite well for this task; you just need to find good hyperparameter values (try a grid search on the `weights` and `n_neighbors` hyperparameters).

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)


In [ ]:
X, y = mnist["data"], mnist["target"]
X.shape
print ((70000, 784))
y.shape
print((70000,))

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
some_digit = X[36000]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap = matplotlib.cm.binary,
interpolation="nearest")
plt.axis("off")
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3, n_jobs=-1)
grid_search.fit(X_train, y_train)


In [ ]:
grid_search.best_params_


In [ ]:
from sklearn.metrics import accuracy_score

y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

**B.** Write a function that can shift an MNIST image in any direction (left, right, up, or down) by one pixel. You can use the shift() function from the scipy.ndimage.interpolation module. For example, shift(image, [2, 1], cval=0) shifts the image two pixels down and one pixel to the right. Then, for each image in the training set, create four shifted copies (one per direction) and add them to the training set. Finally, train your best model on this expanded training set and measure its accuracy on the test set. You should observe that your model performs even better now! This technique of artificially growing the training set is called data augmentation or training set expansion.

In [ ]:
from scipy.ndimage.interpolation import shift

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

In [ ]:
image = X_train[1000]
shifted_image_down = shift_image(image, 0, 5)
shifted_image_left = shift_image(image, -5, 0)

plt.figure(figsize=(12,3))
plt.subplot(131)
plt.title("Original", fontsize=14)
plt.imshow(image.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.subplot(132)
plt.title("Shifted down", fontsize=14)
plt.imshow(shifted_image_down.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.subplot(133)
plt.title("Shifted left", fontsize=14)
plt.imshow(shifted_image_left.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.show()

In [ ]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [ ]:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [ ]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf

In [ ]:
knn_clf.fit(X_train_augmented, y_train_augmented)


In [ ]:

y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)